In [6]:
# # MOUNTING GOOGLE DRIVE
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
import os
import json

## Check spider data

In [8]:
# data_path = '/content/drive/MyDrive/CS 685/cs685_project/spider_data'
data_path = '../spider'
print(os.listdir(data_path))

['.DS_Store', 'database', 'dev.json', 'dev_gold.sql', 'README.txt', 'tables.json', 'train_gold.sql', 'train_others.json', 'train_spider.json']


In [9]:
for file in ["train_spider.json", "train_others.json", "dev.json", "tables.json"]:
    with open(os.path.join(data_path, file)) as inf:
        sql_data = json.load(inf)
    print(f'File - {file}; length - {len(sql_data)}')

    print('##############################')
    print('Sample data')
    print('##############################')
    counter = 0
    for item in sql_data:
      for k, v in item.items():
        print(f"{k} \t\t - {v}")

      print()
      counter += 1
      if counter > 0: break
    print('##############################')
    print()


folder = 'database'
db_dir = os.path.join(data_path, folder)
all_folders = os.listdir(db_dir)
# all_folders.remove('.DS_Store')
print(f'Number of tables in {folder} - {len(all_folders)}')
# for i in sorted():
#     print(i)

File - train_spider.json; length - 7000
##############################
Sample data
##############################
db_id 		 - department_management
query 		 - SELECT count(*) FROM head WHERE age  >  56
query_toks 		 - ['SELECT', 'count', '(', '*', ')', 'FROM', 'head', 'WHERE', 'age', '>', '56']
query_toks_no_value 		 - ['select', 'count', '(', '*', ')', 'from', 'head', 'where', 'age', '>', 'value']
question 		 - How many heads of the departments are older than 56 ?
question_toks 		 - ['How', 'many', 'heads', 'of', 'the', 'departments', 'are', 'older', 'than', '56', '?']
sql 		 - {'from': {'table_units': [['table_unit', 1]], 'conds': []}, 'select': [False, [[3, [0, [0, 0, False], None]]]], 'where': [[False, 3, [0, [0, 10, False], None], 56.0, None]], 'groupBy': [], 'having': [], 'orderBy': [], 'limit': None, 'intersect': None, 'union': None, 'except': None}

##############################

File - train_others.json; length - 1659
##############################
Sample data
################

In [10]:
for file in ["train_spider.json", "train_others.json", "dev.json"]:
# for file in ["dev.json"]:
    with open(os.path.join(data_path, file)) as inf:
        sql_data = json.load(inf)
    print(f'File - {file}; length - {len(sql_data)}')

    print('##############################')
    print('Sample data')
    print('##############################')
    counter = 1
    all_queries = []
    for item in sql_data:
        for k, v in item.items():          
          if k == "query":  all_queries.append(v)

    print('##############################')

    print(f"all_queries - {len(all_queries)}")
    all_queries = list(set(all_queries))
    print(f"all_queries - {len(all_queries)} - after removing duplicate")

    print()
    counts = {}
    check = ['SELECT ', ' FROM ', ' GROUP BY ', ' WHERE ', ' JOIN ', ' MATCH ',
             ' DESC ', ' LIMIT ', ' ORDER BY ', ' HAVING ', ' AS ', ' UNION ',
             ' OR ', ' AND ', ' IN ', ' NOT IN ', ' DISTINCT ', ' max(',
             ' min(', ' avg(', ' count(', ' > ', ' < ', ' = ']
    for idx, query in enumerate(all_queries):
      # print(idx, query)
      for item in check:
        if item in query:
          var_name = item.strip()
          if var_name not in counts.keys():
            counts[var_name] = 0
          counts[var_name] += 1

    print(counts)
    print()

File - train_spider.json; length - 7000
##############################
Sample data
##############################
##############################
all_queries - 7000
all_queries - 3964 - after removing duplicate

{'SELECT': 3964, 'FROM': 3964, 'JOIN': 1565, 'LIMIT': 625, 'ORDER BY': 935, 'AS': 1548, '=': 2142, 'WHERE': 1941, 'IN': 154, 'NOT IN': 128, 'avg(': 265, 'GROUP BY': 1011, 'DESC': 502, 'count(': 1088, 'HAVING': 250, '<': 161, 'AND': 308, '>': 410, 'OR': 126, 'max(': 134, 'min(': 93, 'UNION': 35, 'DISTINCT': 246, 'MATCH': 11}

File - train_others.json; length - 1659
##############################
Sample data
##############################
##############################
all_queries - 1659
all_queries - 750 - after removing duplicate

{'SELECT': 750, 'FROM': 750, 'WHERE': 707, 'JOIN': 482, 'AS': 482, 'DISTINCT': 295, '=': 698, '>': 106, 'AND': 296, 'DESC': 42, 'LIMIT': 45, 'ORDER BY': 86, 'IN': 67, 'GROUP BY': 99, '<': 21, 'HAVING': 20, 'NOT IN': 6}

File - dev.json; length - 1034
#

## Splitting train_spider.json into train and test

### Combine train_spider and train_others

In [11]:
file = "train_spider.json"
with open(os.path.join(data_path, file)) as inf:
    train_spider = json.load(inf)
print(f'File - {file}; length - {len(train_spider)}')

train_others = []
# file = "train_others.json"
# with open(os.path.join(data_path, file)) as inf:
#     train_others = json.load(inf)
# print(f'File - {file}; length - {len(train_others)}')

combined = train_spider + train_others
print(f'combined length - {len(combined)}')

train_size = 0.9
idx = int(len(combined)*train_size)
print(f'idx - {idx}')

train = combined[:idx]
test = combined[idx:]

print(f'File - train; length - {len(train)}')
print(f'File - test; length - {len(test)}')

File - train_spider.json; length - 7000
combined length - 7000
idx - 6300
File - train; length - 6300
File - test; length - 700


### Check for overlap

In [12]:
train_queries = [item['query'] for item in train]

print(f"train queries - {len(train_queries)}")
train_queries_unq = list(set(train_queries))
print(f"train queries - {len(train_queries_unq)} - after removing duplicates")

test_queries = [item['query'] for item in test]

print(f"test queries - {len(test_queries)}")
test_queries_unq = list(set(test_queries))
print(f"test queries - {len(test_queries_unq)} - after removing duplicates")

common_qs = [q for q in test_queries if q in train_queries]
common_qs = list(set(common_qs))
print(f"common queries - {len(common_qs)}")

train queries - 6300
train queries - 3577 - after removing duplicates
test queries - 700
test queries - 389 - after removing duplicates
common queries - 2


In [13]:
remove_items = []
for item1 in test:
  for dup in common_qs:
    if item1['query'] == dup:
      remove_items.append(item1)
  
len(remove_items)

4

In [14]:
for item in remove_items:
  test.remove(item)
  train.append(item)

len(train), len(test)

(6304, 696)

In [15]:
for list_l in [train, test]:
  all_queries = []
  for item in list_l:
      for k, v in item.items():          
        if k == "query":  all_queries.append(v)

  print(f"all_queries - {len(all_queries)}")
  all_queries = list(set(all_queries))
  print(f"all_queries - {len(all_queries)} - after removing duplicate")

  print('##############################')

all_queries - 6304
all_queries - 3577 - after removing duplicate
##############################
all_queries - 696
all_queries - 387 - after removing duplicate
##############################


In [16]:
data_path_target = "/Users/aishwarya/Downloads/spring23/cs685-NLP/project/data/split"

if not os.path.isdir(data_path_target):
    os.makedirs(data_path_target)

# with open(os.path.join(data_path_target, "spider_combined.json"), 'wt') as out:
#   json.dump(combined, out, sort_keys=True, indent=2, separators=(',', ': '))

with open(os.path.join(data_path_target, "spider_train.json"), 'wt') as out:
  json.dump(train, out, sort_keys=True, indent=2, separators=(',', ': '))

with open(os.path.join(data_path_target, "spider_test.json"), 'wt') as out:
  json.dump(test, out, sort_keys=True, indent=2, separators=(',', ': '))


## Check for overlap in Train-Dev-Test dataset